In [1]:
from matrix_factorisation import nmf_features, transform, transform_2, preprocess, get_static_features

In [2]:
import cvxpy as cvx
import numpy as np
import pandas as pd

In [3]:
import  os

import numpy as np
import pandas as pd
import pickle
from features import feature_map

In [4]:
import  os

import numpy as np
import pandas as pd


from create_df_larger import read_df_larger
df, dfc, all_homes, appliance_min, national_average = read_df_larger()

df = df.rename(columns={'house_num_rooms':'num_rooms',
                        'num_occupants':'total_occupants',
                        'difference_ratio_min_max':'ratio_difference_min_max'})
K_min, K_max = 1,6
F_min, F_max=1,8

from all_functions import *
from features_larger import *

In [5]:
import itertools
feature_combinations = [['None']]
for l in range(1,4):
    for a in itertools.combinations(['occ','area','rooms'], l):
        feature_combinations.append(list(a))

In [23]:
#out_overall = {}
#error_overall = {}
#for appliance in ['dw','oven']:
for appliance in ['fridge','hvac','mw','wm']:

    out = {}
    error = {}
    print appliance
    if appliance=="hvac":
        start, end = 5,11
    else:
        start, end=1,13
    X_matrix, X_normalised, col_max, col_min, appliance_cols, aggregate_cols = preprocess(df, dfc, appliance)
    static_features= get_static_features(dfc, X_normalised)
    
    
    
    for feature_comb in np.array(feature_combinations)[:]:
        out[tuple(feature_comb)]={}
        error[tuple(feature_comb)]={}
        
        if 'None' in feature_comb:
            idx_user=None
            data_user=None
        else:
            idx_user = {}
            data_user = {}
            dictionary_static = {}
            for feature in feature_comb:
                dictionary_static[feature]=static_features[feature]
            static_features_df = pd.DataFrame(dictionary_static, index=range(len(X_normalised.index)))


            for fe in static_features_df.columns:
                idx_user[fe]=np.where(static_features_df[fe].notnull())[0]
                data_user[fe]=static_features_df[fe].dropna().values
            
        for lat in range(1,10):
            
            if lat<len(feature_comb):
                continue
            out[tuple(feature_comb)][lat]={}


            for home in X_normalised.index:
                

                X_home = X_normalised.copy()
                for month in range(start, end):
                    X_home.loc[home, '%s_%d' %(appliance, month)] = np.NAN
                mask = X_home.notnull().values
                # Ensure repeatably random problem data.
                A = X_home.copy()
                X, Y, res = nmf_features(A, lat,idx_user, data_user, 10)

                pred_df = pd.DataFrame(Y*X)
                pred_df.columns = X_normalised.columns
                pred_df.index = X_normalised.index
                out[tuple(feature_comb)][lat][home] = transform_2(pred_df.ix[home], appliance, col_max, col_min)[appliance_cols]

            # Evaluation

            gt = X_matrix[appliance_cols]

            pr = pd.DataFrame(out[tuple(feature_comb)][lat]).T

            error_df = (pr-gt).abs()
            error[tuple(feature_comb)][lat] = error_df.div(gt).mul(100).median().mean()
            print error[tuple(feature_comb)][lat], feature_comb, lat
    out_overall[appliance]=out
    error_overall[appliance]=error

fridge
28.3997325236 ['None'] 1
31.9768780113 ['None'] 2
30.8229177565 ['None'] 3
30.9856796647 ['None'] 4
36.7142198273 ['None'] 5
37.069110831 ['None'] 6
39.8378630108 ['None'] 7
41.5939288477 ['None'] 8
38.3064769451 ['None'] 9
34.1280479944 ['occ'] 1
24.7904464353 ['occ'] 2
30.4286952912 ['occ'] 3
28.951977106 ['occ'] 4
33.7381230135 ['occ'] 5
34.4363986101 ['occ'] 6
34.9652639518 ['occ'] 7
39.9654528172 ['occ'] 8
37.566813555 ['occ'] 9
28.1653388835 ['area'] 1
28.7183355178 ['area'] 2
31.4975226403 ['area'] 3
30.600713927 ['area'] 4
35.832572931 ['area'] 5
33.9378583293 ['area'] 6
34.677863696 ['area'] 7
47.353092134 ['area'] 8
41.9533444902 ['area'] 9
29.3970658269 ['rooms'] 1
32.4193738445 ['rooms'] 2
30.6751040581 ['rooms'] 3
30.3118376727 ['rooms'] 4
35.8659054965 ['rooms'] 5
35.986859439 ['rooms'] 6
39.4716580698 ['rooms'] 7
40.587136059 ['rooms'] 8
37.4722493256 ['rooms'] 9
24.747916968 ['occ', 'area'] 2
24.6696879985 ['occ', 'area'] 3
27.2969141989 ['occ', 'area'] 4
30.8456

ValueError: zero-size array to reduction operation

In [37]:
pd.DataFrame(error_overall['fridge'])

None       area                   occ                        \
         NaN        NaN      rooms        NaN       area              
         NaN        NaN        NaN        NaN        NaN      rooms   
1  28.399733  28.165339        NaN  34.128048        NaN        NaN   
2  31.976878  28.718336  29.305906  24.790446  24.747917        NaN   
3  30.822918  31.497523  31.620846  30.428695  24.669688  24.711293   
4  30.985680  30.600714  31.360607  28.951977  27.296914  30.314715   
5  36.714220  35.832573  35.936554  33.738123  30.845663  29.829855   
6  37.069111  33.937858  33.192441  34.436399  29.098348  33.287602   
7  39.837863  34.677864  37.971074  34.965264  30.849290  32.515264   
8  41.593929  47.353092  43.422378  39.965453  38.204025  40.158163   
9  38.306477  41.953344  36.892098  37.566814  40.974347  41.395598   

                  rooms  
       rooms        NaN  
         NaN        NaN  
1        NaN  29.397066  
2  25.990855  32.419374  
3  30.496838  30.675104  
4  29.038388  30.311838  
5  33.415362  35.865905  
6  34.418940  35.986859  
7  34.169627  39.471658  
8  39.449080  40.587136  
9  38.267309  37.472249

In [7]:
%matplotlib inline
pd.DataFrame(error).T.min().min()

48.568681550176741

In [7]:
X_matrix.columns

Index([u'fridge_1', u'fridge_2', u'fridge_3', u'fridge_4', u'fridge_5',
       u'fridge_6', u'fridge_7', u'fridge_8', u'fridge_9', u'fridge_10',
       u'fridge_11', u'fridge_12', u'aggregate_1', u'aggregate_2',
       u'aggregate_3', u'aggregate_4', u'aggregate_5', u'aggregate_6',
       u'aggregate_7', u'aggregate_8', u'aggregate_9', u'aggregate_10',
       u'aggregate_11', u'aggregate_12'],
      dtype='object')

In [26]:
feature_combinations

[['None'],
 ['occ'],
 ['area'],
 ['rooms'],
 ['occ', 'area'],
 ['occ', 'rooms'],
 ['area', 'rooms'],
 ['occ', 'area', 'rooms']]